In [11]:
import pandas as pd 
from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template
import pymongo
import requests

In [13]:
marsurl = 'https://redplanetscience.com/'

In [13]:
jplurl = 'https://spaceimages-mars.com'

In [14]:
marsfactsurl = 'https://galaxyfacts-mars.com'

In [15]:
hemiurl = 'https://marshemispheres.com/'

In [14]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
mars = browser.visit(marsurl)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Zach\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [15]:
def marsurlscrape(): 

 executable_path = {'executable_path': ChromeDriverManager().install()}
 browser = Browser('chrome', **executable_path, headless=False)
 marsurl = 'https://redplanetscience.com/'
 browser.visit(marsurl)

# %%
article_info = []
for x in range(10):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    articles = soup.find_all('div', class_ = "list_text")

    # Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. 
    for article in articles:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        #Assign the text to variables that you can reference later.
        title = article.find('div', class_= "content_title").text
        paragraph = article.find('div', class_ = "article_teaser_body").text
        article_dict = {
            "title" : title, 
            "paragraph" : paragraph
        }
        article_info.append(article_dict)
print(article_info)

[{'title': "NASA's Mars Rover Drivers Need Your Help", 'paragraph': 'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.'}, {'title': "All About the Laser (and Microphone) Atop Mars 2020, NASA's Next Rover", 'paragraph': 'SuperCam is a rock-vaporizing instrument that will help scientists hunt for Mars fossils.'}, {'title': 'Mars Scientists Investigate Ancient Life in Australia', 'paragraph': "Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. "}, {'title': "Meet the People Behind NASA's Perseverance Rover", 'paragraph': "These are the scientists and engineers who built NASA's next Mars rover and who will guide it to a safe landing in Jezero Crater. "}, {'title': 'NASA Adds Return Sample Scientists to Mars 2020 Leadership Team', 'paragraph': 'The leadership co

In [44]:
jpl = browser.visit(jplurl)

In [45]:
featured_image_url = []

In [46]:
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    images = soup.find_all('div', class_="thmb")

    # Use splinter to navigate the site and find the image url for the current Featured Mars Image 
    #and assign the url string to a variable called `featured_image_url`.
    for image in images:
        div = image.find('img')
        src = div['src']
        link = ('https://spaceimages-mars.com/' + src)
        featured_image_url.append(link)
        print(link)
    try:
        browser.links.find_by_partial_text('next').click()
          
    except:
        print("Scraping Complete")

https://spaceimages-mars.com/image/mars/Icaria Fossae7.jpg
https://spaceimages-mars.com/image/mars/Proctor Crater Dunes 7.jpg
https://spaceimages-mars.com/image/mars/Icaria Fossae7.jpg
https://spaceimages-mars.com/image/mars/Proctor Crater Dunes 7.jpg
https://spaceimages-mars.com/image/mars/Proctor Crater Dunes 7.jpg
https://spaceimages-mars.com/image/mars/Icaria Fossae7.jpg
https://spaceimages-mars.com/image/mars/Icaria Fossae.jpg
https://spaceimages-mars.com/image/mars/Ariadnes Colles4.jpg
https://spaceimages-mars.com/image/mars/Niger Vallis.jpg
https://spaceimages-mars.com/image/mars/Proctor Crater Dunes.jpg
https://spaceimages-mars.com/image/mars/Niger Vallis.jpg
https://spaceimages-mars.com/image/mars/Daedalia Planum.jpg
https://spaceimages-mars.com/image/mars/Sirenum Fossae.jpg
https://spaceimages-mars.com/image/mars/Ariadnes Colles4.jpg
https://spaceimages-mars.com/image/mars/South Polar Cap.jpg
https://spaceimages-mars.com/image/mars/Daedalia Planum.jpg
https://spaceimages-mars

In [47]:
urls = []
for i in featured_image_url: 
    j = i.replace(' ','%20')
    urls.append(j)
    
urls = pd.Series([urls])
urls.drop_duplicates(keep='last')
print(urls)

0    [https://spaceimages-mars.com/image/mars/Icari...
dtype: object


In [48]:
mf = browser.visit(marsfactsurl)

In [49]:
# Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) 
tables = pd.read_html(marsfactsurl)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [50]:
#Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
marsdf = tables[1]
marsdf.columns = ['Index', 'Mars']
marsdf

,Index,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [51]:
total_df = tables[0]
total_df.columns = ['Index', 'Mars', 'Earth']
total_df

,Index,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [52]:
#Use Pandas to convert the data to a HTML table string.
totalhtml = total_df.to_html()
marshtml = marsdf.to_html()
text_file = open("index.html", "w")
text_file.write(totalhtml)
text_file.write(marshtml)
text_file.close()

In [53]:
#Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.
hemi = browser.visit(hemiurl)

In [ ]:
#Save both the image url string for the full resolution hemisphere image, 
#and the Hemisphere title containing the hemisphere name. 
#Use a Python dictionary to store the data using the keys `img_url` and `title`.
final_dict= []
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    links = soup.find_all('div', class_="item")

    # Use splinter to navigate the site and find the image url for the current Featured Mars Image 
    #and assign the url string to a variable called `featured_image_url`.
    for link in links:
        img = link.find('div', class_ = "description")
        img_link = img.find('a')
        href = img_link['href']
        img_url = ("https://marshemispheres.com/" + href)
        title = link.find('p').text
        marshemi_dict = {
            "img_url" : img_url,
            "title" : title
        }
        final_dict.append(marshemi_dict)
print(final_dict)

[{'img_url': 'https://marshemispheres.com/cerberus.html', 'title': 'Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…'}, {'img_url': 'https://marshemispheres.com/schiaparelli.html', 'title': 'Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…'}, {'img_url': 'https://marshemispheres.com/syrtis.html', 'title': 'Mosaic of the Syrtis Major hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of about 100 red and violet…'}, {'img_url': 'https://marshemispheres.com/valles.html', 'title': 'Mosaic of the Valles Marineris hemisphere of Mars projected into point perspective, a view similar to that which one would see f